In [ ]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
from pathlib import Path
import torch

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.0f}'.format)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(token=userdata.get('Hugging_Face'))

In [ ]:
# === Load Mistral model ===
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B" #"google/gemma-2b-it" #"mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16
)

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/6.62G [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.61G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
hebrew_tweet = 'אני מצביע ביבי, מה אתם?'

prompt = f"""
Classify bot activity in Israeli elections.

Some tweets are from bots in foreign influence campaigns: they mimic real users and spread polarization and usualy has small comunitys. Others are from authentic citizens.

Tweet:
"{hebrew_tweet}"

- Retweets received: 233.00
- Retweets made: 11.00
- Betweenness centrality (relative): 7.90
- Closeness centrality (relative): 1.19
- Community size: 36277.00
- Bot neighbors: 0.00
- Non-bot neighbors: 244.00

Was this tweet likely written by a bot?

Answer: Yes or No and explain
Answer:
"""

In [ ]:
prompt = """Classify bot activity in Israeli elections.

Some tweets are from bots in foreign influence campaigns: they mimic real users and spread polarization and usualy has small communitys. Others are from authentic citizens.

Tweet:
"#רוץ_לבד_בבחירות #בן_גביר בימים הקרובים ייסגרו לוחות הבחירות ותצטרך להיות עבדו של נתניהו עד השנה הבאה."

Was this tweet likely written by a bot?

Answer (Yes or No):
<fill in here>

Reasoning:
<continue>
"""

In [ ]:
from transformers import pipeline

gemma_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

response = gemma_pipe(prompt, max_new_tokens=100, do_sample=False)[0]["generated_text"]

# Print output (cleaned)
print("🧪 Prompt:\n", prompt)
print("\n🤖 gemma's Response:\n", response.replace(prompt, "").strip())


Device set to use cuda:0


🧪 Prompt:
 Classify bot activity in Israeli elections.

Some tweets are from bots in foreign influence campaigns: they mimic real users and spread polarization and usualy has small communitys. Others are from authentic citizens.

Tweet:
"#רוץ_לבד_בבחירות #בן_גביר בימים הקרובים ייסגרו לוחות הבחירות ותצטרך להיות עבדו של נתניהו עד השנה הבאה."

Was this tweet likely written by a bot?

Answer (Yes or No):
<fill in here>

Reasoning:
<continue>


🤖 gemma's Response:
 </think>

Answer: No

Reasoning: The tweet in question is in Hebrew and uses the hashtag "#-chanut_לבד_בבחירות", which is a Hebrew acronym for "From the beginning of the elections." The content appears to be a call to action for citizens to support Prime Minister Ehud Olmert during the upcoming elections. The username "בן_גביר" translates to "Ben Gvir," which is a real person, and the context of the tweet does not indicate any


In [ ]:
# === Inference function ===
def generate_responses(prompts, batch_size=8, max_new_tokens=10):
    responses = []
    for i in tqdm(range(0, len(prompts), batch_size), desc="Generating responses"):
        batch = prompts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device)

        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,           # Sampling enabled
            temperature=0.6,          # Controls randomness
            top_p=0.95,               # Controls nucleus sampling
            pad_token_id=tokenizer.eos_token_id
        )

        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        responses.extend(decoded)
    return responses

In [ ]:
# === Postprocessing: extract prediction from generated text ===
def extract_prediction(text):
    text = text.strip().lower()

    if "yes" in text and "no" not in text:
        return "Yes"
    elif "no" in text:
        return "No"
    else:
        return "Unclear"

In [ ]:
# === Load prompt data ===
prompts_df = pd.read_csv("/content/drive/MyDrive/Thesis/prompts_for_sampled_data/balanced_bot_prompts_nov2022.csv")

In [ ]:
# === Prompt types to try ===
prompt_types = [
    "Minimal Prompt",
    "Minimal Prompt with CoT",
    "Degree-Based Prompt",
    "Centrality-Based Prompt",
    "Community Prompt",
    "Full-Feature Prompt",
    "Chain-of-Thought Prompt"
]

# === Function to generate single response ===
def run_single_prompt(prompt_text, max_new_tokens=100):
    inputs = tokenizer(prompt_text, return_tensors="pt", return_attention_mask=True, truncation=True, max_length=512).to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,           # Sampling enabled
        temperature=0.6,          # Controls randomness
        top_p=0.95,               # Controls nucleus sampling
        pad_token_id=tokenizer.eos_token_id
    )
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return full_output.replace(prompt_text.strip(), "").strip()  # clean the echoed prompt

# === Try one prompt of each type ===
print("\n🧪 Gemma Responses for One Prompt of Each Type:\n")
for prompt_type in prompt_types:
    prompt_text = prompts_df[prompt_type].dropna().iloc[0]
    print(f"📌 {prompt_type}:\n")
    print("📝 Prompt:")
    print(prompt_text)
    print("\n🤖 Response:")
    print(run_single_prompt(prompt_text))
    print("="*100, "\n")


🧪 Gemma Responses for One Prompt of Each Type:

📌 Minimal Prompt:

📝 Prompt:
‫אתה מומחה בזיהוי מבצעי התערבות זרה בבחירות בישראל. בוטים לרוב משתייכים לקהילות קטנות, מפיצים תוכן מקטב, ולעיתים הטקסטים נראים כמתורגמים משפה זרה.

        ציוץ: "ולא, זה לא בגלל שרוסיה מדינה וחיזבאללה ארגון טרור. זה הבדל תרבותי."
        האם הציוץ נכתב על ידי בוט? ענה רק בכן/לא
        

🤖 Response:
- Explain why you are answering.

Assistant<think>
Okay, so I'm trying to figure out whether the statement "ולא, זה לא בגלל שרוסיה מדינה וחיזבאללה ארגון טרור. זה הבדל תרבותי." was written by a bot. First, I need to understand what the statement is saying. It seems to be arguing against a claim that something is due to Russia's government and instead calling it a political difference.

Looking at the

📌 Minimal Prompt with CoT:

📝 Prompt:
‫אתה מומחה בזיהוי מבצעי התערבות זרה בבחירות בישראל. בוטים לרוב משתייכים לקהילות קטנות, מפיצים תוכן מקטב, ולעיתים הטקסטים נראים כמתורגמים משפה זרה.

        ציוץ: "ולא, זה לא בגלל

KeyboardInterrupt: 

In [ ]:
# === Run on 10 rows ===
print("\n🧪 Gemma Responses for 10 Rows per Prompt Type:\n")
for i in range(10):
    print(f"\n🔢 Tweet #{i+1}")
    for prompt_type in prompt_types:
        prompt_text = prompts_df[prompt_type].dropna().iloc[i]
        print(f"\n📌 {prompt_type}:")
        print("📝 Prompt:")
        print(prompt_text)
        print("\n🤖 Response:")
        print(run_single_prompt(prompt_text))
        print("-" * 80)
    print("=" * 100)


🧪 Gemma Responses for 10 Rows per Prompt Type:


🔢 Tweet #1

📌 Minimal Prompt:
📝 Prompt:
Classify bot activity in Israeli elections.
Some tweets are from bots in foreign influence campaigns: they spread polarization and usually belong to small communities. Others are from authentic citizens.

Tweet:
"ולא, זה לא בגלל שרוסיה מדינה וחיזבאללה ארגון טרור. זה הבדל תרבותי."

Was this tweet written by a bot?
Answer one word (Yes or No):


🤖 Response:
</think>

No

The tweet expresses a political stance against a supposed terror group, which is consistent with the content of many tweets from real citizens. The mention of "הבדל תרבותי" (a political difference) aligns with typical expressions from individuals rather than automated bots.
--------------------------------------------------------------------------------

📌 Minimal Prompt with CoT:
📝 Prompt:
Classify bot activity in Israeli elections.
Some tweets are from bots in foreign influence campaigns: they spread polarization and usually belon